#  Analiza zmian jeziora Aralskiego (2017–2025)
## Sentinel‑2 + Microsoft Planetary Computer + Apache Sedona

**Cel ćwiczenia:**
1) Pobierz dane Sentinel‑2 (Green – `B03`, NIR – `B08`) z Microsoft Planetary Computer dla jeziora Aralskiego — po 1 scenie na każdy rok 2017–2025 (preferencyjnie czerwiec–sierpień, zachmurzenie < 10%).
2) Wczytaj dane do Apache Sedona (Spark) i wykonaj analizę NDWI: `NDWI = (Green - NIR) / (Green + NIR)`.
3) Zwizualizuj zmiany średniego NDWI w czasie.



##  Środowisko i biblioteki
Zalecane biblioteki:
- `pystac-client`, `planetary_computer`, 

Zainstaluj, jeśli potrzeba (np. dla obrazu Dockera)

In [ ]:
!pip install pystac-client
!pip install planetary-computer

## 1) Pobranie danych z Microsoft Planetary Computer
Napisz funkcję `get_aral_s2(years, bbox)`, która dla każdego roku zwróci po 1 najlepszej scenie Sentinel‑2 L2A (najmniejsze zachmurzenie w czerwcu–sierpniu).

**Wskazówki:**
- Katalog STAC: `https://planetarycomputer.microsoft.com/api/stac/v1`
- Kolekcja: `sentinel-2-l2a`
- Filtrowanie: `datetime`, `bbox`, `eo:cloud_cover < 20`, preferuj letnie miesiące.
- Użyj `planetary_computer.sign()` do podpisywania assets.
- Do wczytania pasm użyj `stackstac.stack()`.

In [ ]:
import datetime
import pystac_client
import planetary_computer

In [ ]:
def get_aral_s2(years, bbox):
    """
    Pobiera po jednej najlepszej scenie Sentinel-2 L2A (najmniejsze zachmurzenie, czerwiec-sierpień) dla każdego roku z podanego zakresu i bbox.
    Zwraca listę obiektów STAC.Item.
    """
    
    # Połączenie z katalogiem STAC
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    items = []
    for year in years:
        # Zakres dat: czerwiec-sierpień
        start = f"{year}-06-01"
        end = f"{year}-08-31"
        search = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=bbox,
            datetime=f"{start}/{end}",
            query={"eo:cloud_cover": {"lt": 20}},
            sortby=[{"field": "eo:cloud_cover", "direction": "asc"}]
        )
        # Pobierz najlepszą scenę (najmniejsze zachmurzenie)
        item = next(search.get_items(), None)
        if item:
            # Podpisz assety do pobrania przez planetary_computer
            item = planetary_computer.sign(item)
            items.append(item)
    return items


In [ ]:
# Przykładowe wywołanie funkcji get_aral_s2
# BBOX jeziora Aralskiego (przykładowy, można doprecyzować):
aral_bbox = [45.62, 58.80,  45.67, 58.90]  # [min lon, min lat, max lon, max lat]
years = list(range(2017, 2026))

aral_items = get_aral_s2(years, aral_bbox)
print(f'Pobrano scen: {len(aral_items)}')
for item in aral_items:
    print(item.id, item.datetime, item.properties.get('eo:cloud_cover'))

## 2) Wczytanie do Apache Sedona i przygotowanie danych NDWI
Utwórz sesję Spark + Sedona. Następnie załaduj raster

In [ ]:
!pip install pandas==1.5.3
!pip install stackstac

In [ ]:
from sedona.spark import SedonaContext
from pyspark.sql import SparkSession


# Utwórz sesję Spark + Sedona
spark = SparkSession.builder \
    .appName("aral-s2") \
    .config("spark.jars.packages", ",".join([
        "org.apache.sedona:sedona-spark-shaded-3.5_2.12:1.7.1",
        "org.datasyslab:geotools-wrapper:1.7.0-28.5"
    ])) \
    .getOrCreate()
sedona = SedonaContext.create(spark)
sedona.sparkContext.setLogLevel("ERROR")




In [ ]:
import stackstac
from sedona.spark import SedonaContext
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

spark_dfs = []
for item, year in zip(aral_items, years):
    arr = stackstac.stack(
        [item],
        assets=["B03", "B08"],
        epsg=32642,  
        resolution=60,  
        chunksize=512
    ).isel(x=slice(None, None, 2), y=slice(None, None, 2))  
    arr = arr.squeeze()
    b03 = arr.sel(band="B03").values
    b08 = arr.sel(band="B08").values
    y_coords, x_coords = arr.y.values, arr.x.values
    xx, yy = np.meshgrid(x_coords, y_coords)
    df = pd.DataFrame({
        "x": xx.ravel(),
        "y": yy.ravel(),
        "green": b03.ravel(),
        "nir": b08.ravel(),
        "year": year
    })
    sdf = sedona.createDataFrame(df)
    spark_dfs.append(sdf)


all_spark_df = spark_dfs[0]
for sdf in spark_dfs[1:]:
    all_spark_df = all_spark_df.unionByName(sdf)

all_spark_df.createOrReplaceTempView("aral_ndwi")


## 3) Obliczenie NDWI
Uzyj wzoru `ndwi = (green - nir) / (green + nir)` i policz średni NDWI dla każdego roku.

In [ ]:
ndwi_df = spark.sql("""
    SELECT year, AVG((green - nir) / (green + nir)) AS mean_ndwi
    FROM aral_ndwi
    WHERE green + nir != 0  
    GROUP BY year
    ORDER BY year
""")
ndwi_df.show()


## 4) Wykres zmian NDWI (2017–2025)
Utwórz wykres liniowy średniego NDWI.

In [ ]:
import matplotlib.pyplot as plt
ndwi_pd = ndwi_df.toPandas()
plt.figure(figsize=(8,4))
plt.plot(ndwi_pd['year'], ndwi_pd['mean_ndwi'], marker='o')
plt.xlabel('Rok')
plt.ylabel('Średni NDWI')
plt.title('Zmiany średniego NDWI jeziora Aralskiego (2017–2025)')
plt.grid(True)
plt.show()


## 5) Wizualizacja wyników - wszystkie lata


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for idx, (item, year) in enumerate(zip(aral_items, years)):
    arr = stackstac.stack(
        [item],
        assets=["B03", "B08"],
        epsg=32642,
        resolution=60,
        chunksize=512
    ).isel(x=slice(None, None, 2), y=slice(None, None, 2)).squeeze()
    b03 = arr.sel(band="B03").values.astype(float)
    b08 = arr.sel(band="B08").values.astype(float)
    ndwi = (b03 - b08) / (b03 + b08)
    ax = axes[idx]
    im = ax.imshow(ndwi, cmap='BrBG', vmin=-1, vmax=1)
    ax.set_title(f'NDWI {year}')
    ax.axis('off')

fig.suptitle('NDWI jeziora Aralskiego (2017–2025)', fontsize=18)
cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.05, pad=0.04)
cbar.set_label('NDWI')
plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()